In [ ]:
#dependencias
import pandas as pd
import numpy as np
import re
import unicodedata

#carga de datos
path21 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2021.csv'
path22 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2022.csv'
path23 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2023.csv'
path24 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2024.csv'
path25 = '/home/ubuntu/projects/historical-analysis-management-tickets/data/processed/df2025.csv'


In [45]:
#dataframes de trabajo
df2021 = pd.read_csv(path21, index_col=0)
df2022 = pd.read_csv(path22, index_col=0)
df2023 = pd.read_csv(path23, index_col=0)
df2024 = pd.read_csv(path24, index_col=0)
df2025 = pd.read_csv(path25, index_col=0)

#diccionario de dataframes
dict_df = {
    2021: df2021,
    2022: df2022,
    2023: df2023,
    2024: df2024,
    2025: df2025
}

#estandarizacion de nombres 2.0
for year, df in dict_df.items():
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

In [46]:
#conversion de fechas y calculo de SLA
cols_fecha = ['ultima_modificacion', 'fecha_de_apertura', 'fecha_de_cierre', 'fecha_de_solucion']
cols_str = ['ubicacion', 'seguimientos__origen_de_la_solicitud']

for year, df in dict_df.items():
    print(f"año {year}:")
    for col in cols_fecha:
        print(f"convirtiendo columna {col}")
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
    for col in cols_str:
        print(f"convirtiendo columna {col}")
        df[col] = df[col].astype(str)
    print("----"*10)

año 2021:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2022:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2023:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna fecha_de_cierre
convirtiendo columna fecha_de_solucion
convirtiendo columna ubicacion
convirtiendo columna seguimientos__origen_de_la_solicitud
----------------------------------------
año 2024:
convirtiendo columna ultima_modificacion
convirtiendo columna fecha_de_apertura
convirtiendo columna f

In [ ]:
#arreglar texto con mal formato en estadisticas__hora_de_resolucion
def reparar_formato_tiempo(texto):
    if pd.isna(texto):
        return "0 minutos"
    
    texto = str(texto)
    reparaciones = {
        'Ã\xad': 'í',
        'daas': 'días',
        'dÃas': 'días',
        ' d ': ' días ',
    }
    
    for error, correccion in reparaciones.items():
        texto = texto.replace(error, correccion)
    
    texto = texto.lower().strip()
    
    if 'dias' in texto:
        texto = texto.replace('dias', 'días')
        
    return texto

# Aplicar la limpieza estética a la columna original
for year, df in dict_df.items():
    df['estadisticas__hora_de_resolucion'] = df['estadisticas__hora_de_resolucion'].apply(reparar_formato_tiempo)

In [ ]:
#Transformacion columna de tiempo de resolucion str a fraccion de horas
def limpiar_y_convertir_a_horas(texto):
    if pd.isna(texto) or texto == '':
        return 0.0
    
    texto = str(texto)
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.lower()
    
    dias = re.search(r'(\d+)\s*d', texto)
    horas = re.search(r'(\d+)\s*h', texto)
    minutos = re.search(r'(\d+)\s*m', texto)
    
    d_val = int(dias.group(1)) * 24 if dias else 0
    h_val = int(horas.group(1)) if horas else 0
    m_val = int(minutos.group(1)) / 60 if minutos else 0
    
    return round(d_val + h_val + m_val, 2)

# tranformacion de columna de horas a decimal
col_target = 'estadisticas__hora_de_resolucion'

for year, df in dict_df.items():
    df['resolucion_horas_decimal'] = df[col_target].apply(limpiar_y_convertir_a_horas)

In [ ]:
#eliminar columnas
cols_a_eliminar = ['unnamed_16']

for year, df in dict_df.items():
    df.drop(cols_a_eliminar, axis=1, inplace=True)

In [ ]:
#verificacion de modificaciones
for year, df in dict_df.items():
    print(f"Info del DataFrame {year}:")
    display(df.info())

In [65]:
#Calculo SLA (hipotetico)
OBJETIVO_SLA_HORAS = 24

for year, df in dict_df.items():
    df['cumple_SLA'] = df['resolucion_horas_decimal'] <= OBJETIVO_SLA_HORAS
    porcentaje_cumplimiento = df['cumple_SLA'].mean() * 100
    print(f"El cumplimiento general del SLA del {year}: {porcentaje_cumplimiento:.2f}%")
    print("----"*10)

El cumplimiento general del SLA del 2021: 0.00%
----------------------------------------
El cumplimiento general del SLA del 2022: 39.04%
----------------------------------------
El cumplimiento general del SLA del 2023: 76.76%
----------------------------------------
El cumplimiento general del SLA del 2024: 76.64%
----------------------------------------
El cumplimiento general del SLA del 2025: 81.36%
----------------------------------------


In [ ]:
#tickets con mayor tiempo de resolucion prueba de parametro nuevo
col_target = 'estadisticas__hora_de_resolucion'

for year, df in dict_df.items():
    print(f"Año {year}:")
    df_lentos = df.nlargest(5, 'resolucion_horas_decimal')
    print("Tickets con mayor tiempo de resolución:")
    display(df_lentos[[col_target, 'resolucion_horas_decimal']])


Año 2021:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
0,575 días 15 horas 53 minutos,13815.88
1,559 días 6 horas 17 minutos,13422.28


Año 2022:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
110,604 días 17 horas 50 minutos,14513.83
111,604 días 17 horas 36 minutos,14513.60
112,604 días 16 horas 38 minutos,14512.63
134,225 días 5 horas 9 minutos,5405.15
133,225 días 0 horas 32 minutos,5400.53


Año 2023:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
926,597 días 19 horas 57 minutos,14347.95
929,597 días 17 horas 49 minutos,14345.82
930,596 días 22 horas 36 minutos,14326.60
927,594 días 20 horas 37 minutos,14276.62
928,594 días 20 horas 12 minutos,14276.20


Año 2024:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
5,493 días 15 horas 36 minutos,11847.60
16,471 días 23 horas 52 minutos,11327.87
9,452 días 7 horas 40 minutos,10855.67
6,434 días 22 horas 16 minutos,10438.27
80,422 días 22 horas 26 minutos,10150.43


Año 2025:
Tickets con mayor tiempo de resolución:


,estadisticas__hora_de_resolucion,resolucion_horas_decimal
8225,163 días 3 horas 15 minutos,3915.25
3673,157 días 8 horas 23 minutos,3776.38
7400,153 días 5 horas 23 minutos,3677.38
11356,125 días 20 horas 38 minutos,3020.63
8777,125 días 6 horas 17 minutos,3006.28


In [67]:
#comprobacion de arreglos generados
for year, df in dict_df.items():
    print(f"año {year}:")
    display(df.info())
    print("----"*10)

año 2021:
<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    2 non-null      int64         
 1   titulo                                2 non-null      str           
 2   estado                                2 non-null      str           
 3   ultima_modificacion                   2 non-null      datetime64[us]
 4   fecha_de_apertura                     2 non-null      datetime64[us]
 5   prioridad                             2 non-null      str           
 6   solicitante__solicitante              2 non-null      str           
 7   asignado_a__grupo_de_tecnicos         2 non-null      str           
 8   asignado_a__tecnico                   2 non-null      str           
 9   categoria                             2 non-null      str           
 10  origen_

None

----------------------------------------
año 2022:
<class 'pandas.DataFrame'>
RangeIndex: 3827 entries, 0 to 3826
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    3827 non-null   int64         
 1   titulo                                3827 non-null   str           
 2   estado                                3827 non-null   str           
 3   ultima_modificacion                   3827 non-null   datetime64[us]
 4   fecha_de_apertura                     3827 non-null   datetime64[us]
 5   prioridad                             3827 non-null   str           
 6   solicitante__solicitante              3772 non-null   str           
 7   asignado_a__grupo_de_tecnicos         3561 non-null   str           
 8   asignado_a__tecnico                   2580 non-null   str           
 9   categoria                         

None

----------------------------------------
año 2023:
<class 'pandas.DataFrame'>
RangeIndex: 36259 entries, 0 to 36258
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36259 non-null  int64         
 1   titulo                                36259 non-null  str           
 2   estado                                36259 non-null  str           
 3   ultima_modificacion                   36259 non-null  datetime64[us]
 4   fecha_de_apertura                     36259 non-null  datetime64[us]
 5   prioridad                             36259 non-null  str           
 6   solicitante__solicitante              35899 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33944 non-null  str           
 8   asignado_a__tecnico                   30251 non-null  str           
 9   categoria                       

...

----------------------------------------
año 2024:
<class 'pandas.DataFrame'>
RangeIndex: 35004 entries, 0 to 35003
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    35004 non-null  int64         
 1   titulo                                35004 non-null  str           
 2   estado                                35004 non-null  str           
 3   ultima_modificacion                   35004 non-null  datetime64[us]
 4   fecha_de_apertura                     35004 non-null  datetime64[us]
 5   prioridad                             35004 non-null  str           
 6   solicitante__solicitante              34858 non-null  str           
 7   asignado_a__grupo_de_tecnicos         33346 non-null  str           
 8   asignado_a__tecnico                   33356 non-null  str           
 9   categoria                       

None

----------------------------------------
año 2025:
<class 'pandas.DataFrame'>
RangeIndex: 36527 entries, 0 to 36526
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    36527 non-null  int64         
 1   titulo                                36527 non-null  str           
 2   estado                                36527 non-null  str           
 3   ultima_modificacion                   36527 non-null  datetime64[us]
 4   fecha_de_apertura                     36527 non-null  datetime64[us]
 5   prioridad                             36527 non-null  str           
 6   solicitante__solicitante              36369 non-null  str           
 7   asignado_a__grupo_de_tecnicos         35292 non-null  str           
 8   asignado_a__tecnico                   35160 non-null  str           
 9   categoria                       

None

----------------------------------------
